In [1]:
!pip3 install -U pip setuptools wheel
!pip3 install -U spacy
!python3 -m spacy download xx_sent_ud_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 8.6 MB/s eta 0:00:00:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('xx_sent_ud_sm')


### Import Data & Library 

In [1]:
import pandas as pd
import spacy
import re

In [2]:
df=pd.read_csv('labeled.csv')

In [3]:
df.head()

,Image URL,Timestamp,Filename,OCR_Text,Perusahaan tidak jelas atau tidak ada informasi valid (40%),"Deskripsi posisi, persyaratan, dan lokasi kerja yang tidak jelas (25%)",Deskripsi pekerjaan tidak spesifik (Jobdesc) (20%),Tidak ada proses wawancara dan seleksi yang jelas(5%),Ada permintaan biaya pendaftaran(5%),Gaji tidak realistis (5%),Total,Label
0,https://scontent-xsp2-1.cdninstagram.com/v/t51...,2022-12-28T07:12:37.000Z,2022-12-28_post_lokermagang0_image.jpg,UP WFO MALANG SUGARNUTS INDONESIA PROGRAM MAGA...,"0,00","0,25","0,20","0,00","0,00","0,00","0,45",Not Fraud
1,https://scontent-xsp1-3.cdninstagram.com/v/t51...,2022-12-26T08:03:04.000Z,2022-12-26_post_lokermagang1_combined.jpg,P WFO JAKARTA PSI PSI JAKARTA PROGRAM MAGANG P...,"0,00","0,00","0,20","0,00","0,00","0,00","0,20",Not Fraud
2,https://instagram.fsub8-2.fna.fbcdn.net/v/t51....,2022-12-26T05:05:33.000Z,2022-12-26_post_lokermagang2_combined.jpg,"P WFH FULLTIME Asia Land YES, MUNGKIN INI BUAT...","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Not Fraud
3,https://instagram.fsub8-2.fna.fbcdn.net/v/t51....,2022-12-24T06:54:18.000Z,2022-12-24_post_lokermagang3_image.jpg,P HYBRID SEMARANG hercodigital OPEN INTERNSHIP...,"0,00","0,00","0,20","0,00","0,00","0,00","0,20",Not Fraud
4,https://instagram.fsub8-1.fna.fbcdn.net/v/t51....,2022-12-22T10:00:46.000Z,2022-12-22_post_lokermagang4_image.jpg,UP WFH ONLINE KLTC Anude Jiraorshioifalieaia(i...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00",Not Fraud


### Extraction Company Name 

In [4]:
nlp_multilang = spacy.load('xx_sent_ud_sm')  # Untuk multibahasa
nlp_english = spacy.load('en_core_web_trf')  # Untuk Bahasa Inggris

In [9]:
def is_valid_email(email):
    return re.match(r'\b[\w.-]+@[\w.-]+\.\w{2,}\b', email) and not email.startswith('@')

def extract_company_names(doc):
    return {ent.text for ent in doc.ents if ent.label_ == "ORG"}

In [10]:
def extract_company_from_email_or_text(text):
    emails = re.findall(r'\b[\w.-]+@[\w.-]+\.\w{2,}\b', text)
    extracted_names = set()

    # Mencoba menggunakan model multibahasa dan Bahasa Inggris
    doc_multilang = nlp_multilang(text)
    extracted_names.update(extract_company_names(doc_multilang))

    doc_english = nlp_english(text)
    extracted_names.update(extract_company_names(doc_english))

    # Jika tidak ada nama perusahaan yang ditemukan, ekstrak dari nama dan domain email
    if not extracted_names:
        for email in emails:
            if is_valid_email(email):
                parts = email.split('@')
                username = parts[0]
                domain = parts[1].split('.')[0]
                extracted_names.add(username.capitalize())
                extracted_names.add(domain.capitalize())

    return ", ".join(sorted(extracted_names))

In [11]:
df['Extracted_Companies'] = df['OCR_Text'].apply(extract_company_from_email_or_text)


In [12]:
df.to_csv('semantic_search_.csv',index=False)

In [13]:
df.to_excel('semantic_search.xlsx', index=False)

In [14]:
df.head()

,Image URL,Timestamp,Filename,OCR_Text,Perusahaan tidak jelas atau tidak ada informasi valid (40%),"Deskripsi posisi, persyaratan, dan lokasi kerja yang tidak jelas (25%)",Deskripsi pekerjaan tidak spesifik (Jobdesc) (20%),Tidak ada proses wawancara dan seleksi yang jelas(5%),Ada permintaan biaya pendaftaran(5%),Gaji tidak realistis (5%),Total,Label,Extracted_Companies
0,https://scontent-xsp2-1.cdninstagram.com/v/t51...,2022-12-28T07:12:37.000Z,2022-12-28_post_lokermagang0_image.jpg,UP WFO MALANG SUGARNUTS INDONESIA PROGRAM MAGA...,"0,00","0,25","0,20","0,00","0,00","0,00","0,45",Not Fraud,"Gmail, Sugarnutsco"
1,https://scontent-xsp1-3.cdninstagram.com/v/t51...,2022-12-26T08:03:04.000Z,2022-12-26_post_lokermagang1_combined.jpg,P WFO JAKARTA PSI PSI JAKARTA PROGRAM MAGANG P...,"0,00","0,00","0,20","0,00","0,00","0,00","0,20",Not Fraud,"Opsional, PSI, Program Magang PSI"
2,https://instagram.fsub8-2.fna.fbcdn.net/v/t51....,2022-12-26T05:05:33.000Z,2022-12-26_post_lokermagang2_combined.jpg,"P WFH FULLTIME Asia Land YES, MUNGKIN INI BUAT...","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Not Fraud,Asia Land
3,https://instagram.fsub8-2.fna.fbcdn.net/v/t51....,2022-12-24T06:54:18.000Z,2022-12-24_post_lokermagang3_image.jpg,P HYBRID SEMARANG hercodigital OPEN INTERNSHIP...,"0,00","0,00","0,20","0,00","0,00","0,00","0,20",Not Fraud,"Herco Digital Indonesia, Menguasai, WFO"
4,https://instagram.fsub8-1.fna.fbcdn.net/v/t51....,2022-12-22T10:00:46.000Z,2022-12-22_post_lokermagang4_image.jpg,UP WFH ONLINE KLTC Anude Jiraorshioifalieaia(i...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00",Not Fraud,
